In [27]:
%pip install llama-index-llms-openai
%pip install llama-index-readers-file
%pip install llama-index-embeddings-openai
%pip install llama-index

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/2.0 MB 5.0 MB/s eta 0:00:01
   ----------- ---------------------------- 0.6/2.0 MB 5.9 MB/s eta 0:00:01
   ---------------- ----------------------- 0.8/2.0 MB 5.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.2/2.0 MB 6.5 MB/s eta 0:00:01
   -------------------------------- ------- 1.6/2.0 MB 6.8 MB/s eta 0:00:01
   ------------------------------------ --- 1.8/2.0 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 6.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [24]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)

from llama_index.core.tools import QueryEngineTool, ToolMetadata

In [25]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

In [ ]:
!powershell -command "& {pwd}"
!mkdir "data/10k/"
!powershell -command "& {Invoke-WebRequest 'https://s23.q4cdn.com/407969754/files/doc_financials/2022/ar/2021-Annual-Report.pdf' -OutFile 'data/10k/uber_2021.pdf'}"
!powershell -command "& {Invoke-WebRequest 'https://s27.q4cdn.com/263799617/files/doc_financials/2021/AR/Lyft-Annual-Report-2021.pdf' -OutFile 'data/10k/lyft_2021.pdf'}"

In [58]:
# set openai api key as environment variable
import os
os.environ["OPENAI_API_KEY"] = "sk-API-KEY"

if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./data/10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./data/10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs)
    uber_index = VectorStoreIndex.from_documents(uber_docs)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

In [33]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

c:\Users\rudi\.conda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\rudi\.conda\envs\llm\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rudi\AppData\Local\llama_index\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer

In [91]:
# if not index_loaded:
#     # load data
#     lyft_docs = SimpleDirectoryReader(
#         input_files=["./data/10k/lyft_2021.pdf"]
#     ).load_data()
#     uber_docs = SimpleDirectoryReader(
#         input_files=["./data/10k/uber_2021.pdf"]
#     ).load_data()

#     # build index
#     lyft_index = VectorStoreIndex.from_documents(lyft_docs)
#     uber_index = VectorStoreIndex.from_documents(uber_docs)

#     # persist index
#     lyft_index.storage_context.persist(persist_dir="./storage/lyft")
#     uber_index.storage_context.persist(persist_dir="./storage/uber")

if not index_loaded:
    # load data
    transunet_docs = SimpleDirectoryReader(
        input_files=[r"C:\Users\rudi\Documents\tmp\2102.04306.pdf"]
    ).load_data()

    # build index
    transunet_docs_index = VectorStoreIndex.from_documents(transunet_docs)

    # persist index
    transunet_docs_index.storage_context.persist(persist_dir="./storage/transunet_docs")

In [81]:
# lyft_engine = lyft_index.as_query_engine(similarity_top_k=10)
# uber_engine = uber_index.as_query_engine(similarity_top_k=10)

transunet_engine = transunet_docs_index.as_query_engine(similarity_top_k=10)

In [92]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=transunet_engine,
        metadata=ToolMetadata(
            name="transunet_docs",
            description=(
                "Provides information about the transunet paper. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    # QueryEngineTool(
    #     query_engine=uber_engine,
    #     metadata=ToolMetadata(
    #         name="uber_10k",
    #         description=(
    #             "Provides information about Uber financials for year 2021. "
    #             "Use a detailed plain text question as input to the tool."
    #         ),
    #     ),
    # ),
]

In [51]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

In [84]:
llm = OpenAI(odel="gpt-3.5-turbo-instruct")

agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    # context=context
)

In [93]:
response = agent.chat("What is the transunet paper about? For what is it used?")
print(response)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: transunet_docs
Action Input: {'input': 'What is the transunet paper about? For what is it used?'}
Observation: The TransUNet paper explores the fusion of Transformers and U-Net for medical image segmentation applications. It introduces TransUNet as a robust solution for medical image segmentation by merging the broad self-attention capabilities of Transformers with the accurate localization skills of U-Net. TransUNet is designed to enhance segmentation precision and maintain detailed information in medical images, making it a valuable tool for tasks like multi-organ segmentation and cardiac segmentation in healthcare settings.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The TransUNet paper introduces a framework that combines Transformers and U-Net for medical image segmentation tasks. It is used to improve segmentati

In [94]:
response = agent.chat("Tell more about its architecture. Especially about the transformer and unet parts.")
print(response)

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The TransUNet architecture combines the Transformer model for global context encoding with the U-Net model for spatial information recovery and detail enhancement. The encoder part of TransUNet utilizes self-attention mechanisms from Transformers to capture long-range dependencies, while the decoder refines the segmentation masks using the U-Net structure. This hybrid architecture allows TransUNet to effectively integrate global and local information, leading to improved performance in medical image segmentation tasks.
The TransUNet architecture combines the Transformer model for global context encoding with the U-Net model for spatial information recovery and detail enhancement. The encoder part of TransUNet utilizes self-attention mechanisms from Transformers to capture long-range dependencies, while the decoder refines the segmentation masks using the U-Net structure. This hybrid archi

In [96]:
response = agent.chat("How is the paper structured? What are the main sections?")
print(response)

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: transunet_docs
Action Input: {'input': 'How is the paper structured? What are the main sections?'}
Observation: The paper is structured into several sections. The main sections include:
1. Introduction
2. Background and Related Work
3. Methodology
4. Experiments and Results
5. Discussion
6. Conclusion
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The TransUNet paper is structured into several main sections, including Introduction, Background and Related Work, Methodology, Experiments and Results, Discussion, and Conclusion. Each section provides valuable information about the motivation, implementation, and evaluation of the TransUNet framework for medical image segmentation tasks.
The TransUNet paper is structured into several main sections, including Introduction, Background and Related Work, Methodology, Experiments 

In [97]:
response = agent.chat("Summarize the conclusion of the paper. What are the main findings?")
print(response)

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The conclusion of the TransUNet paper highlights the effectiveness of the proposed framework in improving medical image segmentation tasks by combining the strengths of Transformers and U-Net. The main findings include the superior performance of TransUNet compared to other methods in various medical applications, showcasing the importance of integrating global context encoding and spatial information recovery for accurate segmentation results.
The conclusion of the TransUNet paper highlights the effectiveness of the proposed framework in improving medical image segmentation tasks by combining the strengths of Transformers and U-Net. The main findings include the superior performance of TransUNet compared to other methods in various medical applications, showcasing the importance of integrating global context encoding and spatial information recovery for accurate segmentation results.


In [78]:
response = agent.chat("What was Lyft's revenue growth in 2021?")
print(str(response))

Thought: I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue grew by $843,642,000 in 2021 compared to the previous year.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Lyft's revenue growth in 2021 was $843,642,000 compared to the previous year.
Lyft's revenue growth in 2021 was $843,642,000 compared to the previous year.


In [79]:
response = agent.chat("Compare Lyft's revenue growth to Uber's revenue growth in 2021.")
print(str(response))

Thought: I need to compare the revenue growth of Lyft and Uber in 2021.
Action: lyft_10k
Action Input: {'input': "What was Lyft's revenue growth in 2021?"}
Observation: Lyft's revenue grew by $843,642,000 in 2021 compared to the previous year.
Thought: I have the information about Lyft's revenue growth in 2021. Now, I need to retrieve the information about Uber's revenue growth in 2021.
Action: uber_10k
Action Input: {'input': "What was Uber's revenue growth in 2021?"}
Observation: The provided context information does not contain specific details about Uber's revenue growth in 2021.
Thought: I cannot answer the question with the provided tools.
Answer: I cannot answer the question with the provided tools.
I cannot answer the question with the provided tools.


In [73]:
response = agent.chat(
    "What can you tell me about the financial health of Uber?"
)
print(str(response))

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: uber_10k
Action Input: {'input': "Provide an overview of Uber's financial health in 2021"}
Observation: Uber's financial health in 2021 can be assessed positively based on the information provided in the context. The company's common stock is traded on the New York Stock Exchange under the symbol "UBER," indicating its presence in the public market. Additionally, Uber's annual report on Form 10-K was filed with the Securities and Exchange Commission, showcasing transparency in financial reporting. The company's independent public registered accounting firm is PricewaterhouseCoopers LLP, a reputable firm known for its auditing services. These factors suggest that Uber is committed to financial transparency and accountability, which are essential elements for a healthy financial standing.
Thought: I can answer without using any more tools. I'll use the user's language to an

In [74]:
response = agent.chat(
    "summarize the uber 10k report for me."
)
print(str(response))

Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The Uber 10-K report for 2021 provides detailed information about the company's financial performance, risks, and strategies. It includes insights into Uber's revenue, expenses, profitability, and overall financial health for the year.
The Uber 10-K report for 2021 provides detailed information about the company's financial performance, risks, and strategies. It includes insights into Uber's revenue, expenses, profitability, and overall financial health for the year.


In [69]:
response = agent.chat(
    "summarize the financial performance of Uber in 2021 and provide an analysis"
)
print(str(response))

Thought: The current language of the user is English. I need to use a tool to help me answer the question.
Action: uber_10k
Action Input: {'input': "Please provide a summary of Uber's financial performance in 2021."}
Observation: Uber's financial performance in 2021 can be obtained by requesting a copy of the Company's annual report on Form 10-K filed with the Securities and Exchange Commission. The report will provide detailed information on Uber's financial results for the year.
Thought: I cannot answer the question with the provided tools.
Answer: Uber's financial performance in 2021 can be obtained by requesting a copy of the Company's annual report on Form 10-K filed with the Securities and Exchange Commission. The report will provide detailed information on Uber's financial results for the year.
Uber's financial performance in 2021 can be obtained by requesting a copy of the Company's annual report on Form 10-K filed with the Securities and Exchange Commission. The report will pr